# Visualización con Seaborn

Matplotlib ha demostrado ser una herramienta de visualización increíblemente útil y popular, pero incluso los usuarios más ávidos admitirán que a menudo deja mucho que desear.
Hay varias quejas válidas sobre Matplotlib que surgen a menudo:

- Antes de la versión 2.0, los valores por defecto de Matplotlib no son exactamente las mejores opciones. Se basó en MATLAB alrededor de 1999, y esto a menudo se nota.
- La API de Matplotlib es relativamente de bajo nivel. Hacer una visualización estadística sofisticada es posible, pero a menudo requiere *mucho* código repetitivo.
- Matplotlib es anterior a Pandas por más de una década, y por lo tanto no está diseñado para su uso con Pandas ``DataFrame``s. Para visualizar los datos de un ``DataFrame`` de Pandas, hay que extraer cada ``Serie`` y a menudo concatenarlas en el formato adecuado. Sería mejor tener una librería de ploteo que pudiera usar inteligentemente las etiquetas del ``DataFrame`` en un ploteo.

Una respuesta a estos problemas es [Seaborn](http://seaborn.pydata.org/). Seaborn proporciona una API sobre Matplotlib que ofrece opciones sanas para el estilo de trazado y los colores por defecto, define funciones simples de alto nivel para los tipos de trazado estadísticos comunes, y se integra con la funcionalidad proporcionada por Pandas ``DataFrame``.

## Seaborn Vs Matplotlib

He aquí un ejemplo de un simple gráfico de paseo aleatorio en Matplotlib, utilizando su clásico formato y colores.
Comenzamos con las importaciones típicas:

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
import numpy as np
import pandas as pd

Ahora creamos algunos datos aleatorios:

In [ ]:
# Crear algunos datos
rng = np.random.RandomState(0)
x = np.linspace(0, 10, 500)
y = np.cumsum(rng.randn(500, 6), 0)

Y hacer una trama sencilla:

In [ ]:
# Trazar los datos con los valores por defecto de Matplotlib
plt.plot(x, y)
plt.legend('ABCDEF', ncol=2, loc='upper left');

Aunque el resultado contiene toda la información que nos gustaría que transmitiera, lo hace de una forma que no es del todo agradable estéticamente, e incluso parece un poco anticuada en el contexto de la visualización de datos del siglo XXI.

Veamos ahora cómo funciona con Seaborn.

Como veremos, Seaborn tiene muchas de sus propias rutinas de trazado de alto nivel, pero también puede sobrescribir los parámetros por defecto de Matplotlib y, a su vez, hacer que incluso simples scripts de Matplotlib produzcan resultados muy superiores.
Podemos establecer el estilo llamando al método ``set_theme()`` de Seaborn.
Por convención, Seaborn se importa como ``sns``:

In [ ]:
import seaborn as sns
sns.set_theme()

Now let's rerun the same two lines as before:

In [ ]:
# mismo código de trazado que el anterior
plt.plot(x, y)
plt.legend('ABCDEF', ncol=2, loc='upper left');

## Explorando los plots Seaborn

La idea principal de Seaborn es que proporciona comandos de alto nivel para crear una variedad de tipos de gráficos útiles para la exploración de datos estadísticos, e incluso para el ajuste de modelos estadísticos.

Veamos algunos de los conjuntos de datos y tipos de gráficos disponibles en Seaborn. Tenga en cuenta que todo lo siguiente *podría* hacerse utilizando comandos Matplotlib sin procesar (esto es, de hecho, lo que Seaborn hace bajo el capó), pero la API de Seaborn es mucho más conveniente.

### Histogramas, KDE y densidades

A menudo, en la visualización de datos estadísticos, todo lo que quieres es trazar histogramas y distribuciones conjuntas de variables.
Hemos visto que esto es relativamente sencillo en Matplotlib:

In [ ]:
data = np.random.multivariate_normal([0, 0], [[5, 2], [2, 2]], size=2000)
data = pd.DataFrame(data, columns=['x', 'y'])

for col in 'xy':
    plt.hist(data[col], density=True, alpha=0.5)

En lugar de un histograma, podemos obtener una estimación suave de la distribución utilizando una estimación de la densidad del núcleo, lo que Seaborn hace con ``sns.kdeplot``:

In [ ]:
for col in 'xy':
    sns.kdeplot(data[col], fill=True)

Los histogramas y los KDE pueden combinarse

In [ ]:
sns.histplot(data['x'],kde=True)
sns.histplot(data['y'],kde=True);

Si pasamos el conjunto de datos bidimensional completo a ``kdeplot``, obtendremos una visualización bidimensional de los datos:

In [ ]:
sns.kdeplot(data);

Podemos ver la distribución conjunta y las distribuciones marginales juntas utilizando ``sns.jointplot``.
Para este gráfico, vamos a establecer el estilo a un fondo blanco:

In [ ]:
with sns.axes_style('white'):
    sns.jointplot(data[['x','y']], kind='kde');

### Pair plots

Cuando se generalizan los gráficos conjuntos a conjuntos de datos de mayores dimensiones, se obtienen *gráficos de pares*. Esto resulta muy útil para explorar correlaciones entre datos multidimensionales, cuando se desea representar todos los pares de valores entre sí.

Haremos una demostración con el conocido conjunto de datos Iris, que contiene medidas de pétalos y sépalos de tres especies de iris:

In [ ]:
iris = sns.load_dataset("iris")
iris.head()

Visualizing the multidimensional relationships among the samples is as easy as calling ``sns.pairplot``:

In [ ]:
sns.pairplot(iris, hue='species', height=2.5);

### Histogramas facetados

A veces la mejor manera de ver los datos es a través de histogramas de subconjuntos. El ``FacetGrid`` de Seaborn lo hace extremadamente sencillo.
Vamos a ver unos datos que muestran la cantidad que el personal de un restaurante recibe en propinas en función de varios datos indicadores:

In [ ]:
tips = sns.load_dataset('tips')
tips.head()

In [ ]:
tips['tip_pct'] = 100 * tips['tip'] / tips['total_bill']

grid = sns.FacetGrid(tips, row="sex", col="time", margin_titles=True)
grid.map(plt.hist, "tip_pct", bins=np.linspace(0, 40, 15));

### Joint distributions

De forma similar al pairplot que vimos anteriormente, podemos utilizar ``sns.jointplot`` para mostrar la distribución conjunta entre diferentes conjuntos de datos, junto con las distribuciones marginales asociadas:

In [ ]:
with sns.axes_style('white'):
    sns.jointplot(data=tips,x="total_bill", y="tip",  kind='hex')

El gráfico conjunto puede incluso realizar una estimación automática de la densidad del núcleo y una regresión:

In [ ]:
sns.jointplot(data=tips,x="total_bill", y="tip",  kind='reg')

### Bar plots

In [ ]:
planets = sns.load_dataset('planets')
planets.head()

In [ ]:
with sns.axes_style('white'):
    g = sns.catplot(data=planets,x="year",  aspect=2,
                       kind="count", color='steelblue')
    g.set_xticklabels(step=5)

Podemos aprender más observando el *método* de descubrimiento de cada uno de estos planetas:

In [ ]:
with sns.axes_style('white'):
    g = sns.catplot(data=planets,x="year", aspect=4.0, kind='count',
                       hue='method', order=range(2001, 2015))
    g.set_ylabels('Number of Planets Discovered')

## Ejemplo: Exploración de los tiempos de llegada al maratón

Aquí vamos a ver el uso de Seaborn para ayudar a visualizar y comprender los resultados finales de un maratón.

In [ ]:
# !curl -o data/marathon-data.csv https://raw.githubusercontent.com/jakevdp/marathon-data/master/marathon-data.csv

In [ ]:
data = pd.read_csv('data/marathon-data.csv')
data.head()

Por defecto, Pandas carga las columnas de tiempo como cadenas de Python (tipo ``object``); podemos ver esto mirando el atributo ``dtypes`` del DataFrame:

In [ ]:
data.dtypes

Arreglemos esto proporcionando un conversor para los tiempos:

In [ ]:
import datetime

def convert_time(s):
    h, m, s = map(int, s.split(':'))
    return datetime.timedelta(hours=h, minutes=m, seconds=s)

data = pd.read_csv('data/marathon-data.csv',
                   converters={'split':convert_time, 'final':convert_time})
data.head()

In [ ]:
data.dtypes

Así queda mucho mejor. Para el propósito de nuestras utilidades de trazado Seaborn, vamos a añadir columnas que dan los tiempos en segundos:

In [ ]:
data['split_sec'] = data['split'].astype(int) / 1E9
data['final_sec'] = data['final'].astype(int) / 1E9
data.head()

Para hacernos una idea del aspecto de los datos, podemos trazar un ``jointplot`` sobre los datos:

In [ ]:
with sns.axes_style('white'):
    g = sns.jointplot( data,x="split_sec", y="final_sec", kind='hex')
    g.ax_joint.plot(np.linspace(4000, 16000),
                    np.linspace(8000, 32000), ':k')

La línea de puntos muestra dónde se situaría el tiempo de alguien que corriera el maratón a un ritmo perfectamente constante. El hecho de que la distribución se sitúe por encima de esta línea indica (como era de esperar) que la mayoría de la gente va más despacio a lo largo del maratón.
Si has corrido en competición, sabrás que los que hacen lo contrario -correr más rápido durante la segunda mitad de la carrera- se dice que han hecho un "parcial negativo".

Creemos otra columna en los datos, la fracción fraccionada, que mide el grado en que cada corredor fracciona negativa o positivamente la carrera:

In [ ]:
data['split_frac'] = 1 - 2 * data['split_sec'] / data['final_sec']
data.head()

Cuando esta diferencia de división es menor que cero, la persona divide negativamente la carrera en esa fracción.
Hagamos un gráfico de distribución de esta fracción de división:

In [ ]:
sns.histplot(data['split_frac'], kde=False);
plt.axvline(0, color="k", linestyle="--");

In [ ]:
sum(data.split_frac < 0)

De los casi 40.000 participantes, sólo 250 hicieron un split negativo en su maratón.

Veamos si existe alguna correlación entre esta fracción de parciales y otras variables. Para ello utilizaremos ``pairgrid``, que dibuja gráficos de todas estas correlaciones:

In [ ]:
g = sns.PairGrid(data, vars=['age', 'split_sec', 'final_sec', 'split_frac'],
                 hue='gender')
g.map(plt.scatter, alpha=0.8)
g.add_legend();

Parece que la fracción de división no se correlaciona especialmente con la edad, pero sí con el tiempo final: los corredores más rápidos tienden a tener divisiones más cercanas a la par en su tiempo de maratón.

La diferencia entre hombres y mujeres es interesante. Veamos el histograma de fracciones divididas para estos dos grupos:

In [ ]:
sns.kdeplot(data.split_frac[data.gender=='M'], label='men', fill=True)
sns.kdeplot(data.split_frac[data.gender=='W'], label='women', fill=True)
plt.xlabel('split_frac');

Lo interesante aquí es que hay muchos más hombres que mujeres que corren casi a partes iguales.
Casi parece una distribución bimodal entre hombres y mujeres. Veamos si podemos averiguar lo que ocurre observando las distribuciones en función de la edad.

Una buena forma de comparar distribuciones es utilizar un *violin plot*.

In [ ]:
sns.violinplot(data=data, x="gender", y="split_frac", 
               hue="gender", palette=["lightblue", "lightpink"], legend=False);

Esta es otra forma de comparar las distribuciones entre hombres y mujeres.

Profundicemos un poco más y comparemos estos gráficos de violín en función de la edad. Empezaremos creando una nueva columna en la matriz que especifique la década de edad en la que se encuentra cada persona:

In [ ]:
data['age_dec'] = data.age.map(lambda age: 10 * (age // 10))
data.head()

In [ ]:
men = (data.gender == 'M')
women = (data.gender == 'W')

with sns.axes_style(style=None):
    sns.violinplot(x="age_dec", y="split_frac", hue="gender", data=data,
                   split=True, inner="quartile",
                   palette=["lightblue", "lightpink"])


Si observamos esto, podemos ver en qué difieren las distribuciones de hombres y mujeres: las distribuciones de las divisiones de los hombres de entre 20 y 50 años muestran una pronunciada sobredensidad hacia las divisiones más bajas en comparación con las mujeres de la misma edad (o de cualquier edad, para el caso).

También resulta sorprendente que las mujeres de 80 años parezcan superar a *todos* en términos de tiempo de split. Esto se debe probablemente al hecho de que estamos estimando la distribución a partir de números pequeños, ya que sólo hay un puñado de corredoras en ese rango:

In [ ]:
(data.age > 80).sum()

Volviendo a los hombres con parciales negativos: ¿quiénes son estos corredores? ¿Esta fracción se correlaciona con terminar rápido? Podemos representar esto muy fácilmente. Utilizaremos ``regplot``, que ajustará automáticamente una regresión lineal a los datos:

In [ ]:
g = sns.lmplot(data=data,x='final_sec', y='split_frac', col='gender', 
               markers=".", scatter_kws=dict(color='c'))
g.map(plt.axhline, y=0.1, color="k", ls=":");

Al parecer, los corredores de élite que terminan la carrera en menos de 15.000 segundos, es decir, en unas 4 horas, son los que tienen parciales más rápidos. Las personas más lentas tienen muchas menos probabilidades de tener un segundo parcial rápido (estrategia de carrera en la cual un corredor completa la segunda mitad de la carrera más rápido que la primera mitad.).

<!--NAVIGATION-->
< [Texto y Anotaciones](8-Text_y_Annotation.ipynb) | [Otras Librerías](10-Otras_librerias.ipynb) >

